In [73]:
from datetime import datetime, timedelta
import random

In [74]:
professor = ["임태수 교수님","김상근 교수님","한경수 교수님","최정열 교수님",
             "임상순 교수님", "강영명 교수님","심미나 교수님","박미옥 교수님",
             "고혜경 교수님","임석진 교수님","임황규 교수님"]

job = ["께서 소개해주신 인턴", "께서 추천해주신 부트캠프", "과 함께하는 논문작성", 
       "과 함께하는 학술대회", "의 학습튜터", "과 함께하는 연구", "께서 개최하시는 코딩테스트", 
       "의 직무특강", "이 추천하시는 자격증 준비", "과 함께하는 해외연수"]

plan = []

for loving in professor:
    for doing in job:
        plan.append(loving+doing)
        
def get_month_day_from_integer(day_number):
    start_date = datetime(2024, 1, 1)
    target_date = start_date + timedelta(days=day_number - 1)
    month = target_date.month
    day = target_date.day
    return month, day

In [91]:
random.shuffle(plan)

def random_schedule(N, v):
    random.seed()
    for i in range(N):
        s = random.randint(0, 299)
        e = random.randint(s, s+65)
        v[i] = (s,
                e,
                (e-s)*random.randint(800, 1200),
                random.randint(1, 99))

def input_schedule(N, v):
    print("***이벤트 입력(시작일 종료일 경험치 성공확률*1000)***")
    for i in range(N):
        print(f"{i+1}번째 이벤트 입력 :", end=" ")
        v[i] = tuple(map(int, input().split()))

def top_down_dp(n, end_time, v, schedule):
    if n == N:
        return (0, -1)
    if schedule[end_time][n][0] != 0:
        return schedule[end_time][n]
    if end_time > v[n][0]:
        schedule[end_time][n] = top_down_dp(n+1, end_time, v, schedule)
        return schedule[end_time][n]
    a = top_down_dp(n+1, end_time, v, schedule)
    b = top_down_dp(n+1, v[n][1], v, schedule)
#     b = (b[0] + v[n][2], n)
    b = (b[0] + (v[n][2]*v[n][3]), n)
    schedule[end_time][n] = max(a, b)
    return schedule[end_time][n]

choice = int(input("랜덤 : 1, 선택 : 2\n"))
N = int(input("이벤트 개수 입력\n"))
v = [(0, 0, 0, 0) for _ in range(N)]

if choice == 1:
    random_schedule(N, v)
elif choice == 2:
    input_schedule(N, v)

schedule = [[(0, -1)] * N for _ in range(365)]

v.sort()
print()
print("올해 신청가능한 활동 리스트\n--------------------------------------------------")
for i in range(N):
    s_month, s_day = get_month_day_from_integer(v[i][0])
    e_month, e_day = get_month_day_from_integer(v[i][1])
    print(plan[i]+"\n활동기간 : ", s_month, "/", s_day ," ~ ", e_month, "/", e_day,", 경험치 : ",v[i][2]," ,합격확률 : ",v[i][3],"%\n",sep="")
# for event in v:
#     print(*event)

s = top_down_dp(0, 0, v, schedule)
# print(s[0]//100, s[1])
i = s[1]

print(f"\n\n경험치를 최대로 만들며 겹치지 않는 활동 리스트\n최대 경험치: {s[0]//100}\n--------------------------------------------------")
idx = 1
while i != -1:
    s_month, s_day = get_month_day_from_integer(v[i][0])
    e_month, e_day = get_month_day_from_integer(v[i][1])
    print(idx, end='. ')
    idx += 1
    print(plan[i]," (", s_month, "/", s_day, "~", e_month, "/", e_day,")", sep="")
    day = v[i][1]
    if i==N-1:
        break
    i = schedule[day][i+1][1]
    

랜덤 : 1, 선택 : 2
1
이벤트 개수 입력
10

올해 신청가능한 활동 리스트
--------------------------------------------------
강영명 교수님께서 소개해주신 인턴
활동기간 : 1/6 ~ 1/17, 경험치 : 10978 ,합격확률 : 75%

최정열 교수님과 함께하는 학술대회
활동기간 : 3/31 ~ 4/26, 경험치 : 21944 ,합격확률 : 75%

한경수 교수님께서 소개해주신 인턴
활동기간 : 4/18 ~ 6/4, 경험치 : 43005 ,합격확률 : 83%

고혜경 교수님과 함께하는 학술대회
활동기간 : 5/20 ~ 6/20, 경험치 : 35929 ,합격확률 : 5%

강영명 교수님과 함께하는 연구
활동기간 : 6/8 ~ 7/3, 경험치 : 29200 ,합격확률 : 45%

임태수 교수님께서 개최하시는 코딩테스트
활동기간 : 6/26 ~ 8/27, 경험치 : 62744 ,합격확률 : 46%

고혜경 교수님께서 소개해주신 인턴
활동기간 : 7/16 ~ 8/2, 경험치 : 17357 ,합격확률 : 49%

심미나 교수님께서 추천해주신 부트캠프
활동기간 : 7/16 ~ 8/20, 경험치 : 40600 ,합격확률 : 79%

박미옥 교수님이 추천하시는 자격증 준비
활동기간 : 9/15 ~ 11/1, 경험치 : 46154 ,합격확률 : 87%

임태수 교수님께서 소개해주신 인턴
활동기간 : 9/19 ~ 11/23, 경험치 : 77415 ,합격확률 : 73%



경험치를 최대로 만들며 겹치지 않는 활동 리스트
최대 경험치: 145654
--------------------------------------------------
1. 강영명 교수님께서 소개해주신 인턴 (1/6~1/17)
2. 한경수 교수님께서 소개해주신 인턴 (4/18~6/4)
3. 강영명 교수님과 함께하는 연구 (6/8~7/3)
4. 심미나 교수님께서 추천해주신 부트캠프 (7/16~8/20)
5. 임태수 교수님께서 소개해주신 인턴 (9/19~11/23)